In [1]:
import json
import pandas as ps

# Reading data

In [2]:
with open('../data/new_headers.json', 'r') as in_file:
    json_data = json.load(in_file)
    
def print_diction(some_dict: dict):
    print(json.dumps(some_dict, indent=4, sort_keys=True))

# print_diction(json_data)

In [3]:
titles = dict()
headers = dict()
sub_headers = dict()

def add_tolist_onto_dict(some_dictionary: dict, key, value):
    if key in some_dictionary:
        some_dictionary[key].append(value)
    else:
        some_dictionary[key] = [value]

for filename, file_data in json_data['files'].items():
    add_tolist_onto_dict(titles, filename, file_data['title'])
    for item in file_data['headers']:
        flag, data = item
        if flag == 'he':
            add_tolist_onto_dict(headers, filename, data)
        elif flag == 'sub-he':
            add_tolist_onto_dict(sub_headers, filename, data)

# print_diction(titles)
# print_diction(headers)
# print_diction(sub_headers)

In [8]:
X, y = [], []

def add_to_x_y(d: dict, flag: int):
    for _, lst in d.items():
        for item in lst:
            X.append(item)
            y.append(flag)
    
add_to_x_y(titles, 0)
add_to_x_y(headers, 1)
add_to_x_y(sub_headers, 2)
        
data = ps.DataFrame({'Text': X, 'Flag': y})
data.head(15)

,Flag,Text
0,0,AMENDED AND RESTATED\nBYLAWS OF\nIMPERIAL CAPI...
1,0,CHANGE IN CONTROL AGREEMENT
2,0,PRESCRIPTION DRUG DISCOUNT CARD AGREEMENT
3,0,VOTING AGREEMENT
4,0,SECURED CONVERTIBLE REDEEMABLE DEBENTURE
5,0,FORM OF INDEMNIFICATION AGREEMENT
6,0,Re: Offer of Employment with TranS1 Inc.
7,0,MANAGEMENT SUBSCRIPTION AND STOCK PURCHASE AGR...
8,0,RESTRICTED SHARE UNITS AWARD AGREEMENT
9,1,ARTICLE I\nOFFICES


In [5]:
data.Text = data.Text.apply(lambda x: x.lower())
data.Text = data.Text.apply(lambda x: x.replace('\n', ' '))

data.head(5)

,Flag,Text
0,0,amended and restated bylaws of imperial capita...
1,0,change in control agreement
2,0,prescription drug discount card agreement
3,0,voting agreement
4,0,secured convertible redeemable debenture


# Model

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(data.Text, data.Flag,
                                                    test_size=0.3, random_state=11)

## Naive Bayes

In [11]:
mnb_text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('mnb-clf', MultinomialNB())])

mnb_text_clf = text_clf.fit(X_train, y_train)
mnb_text_clf.score(X_test, y_test)

0.82524271844660191

In [14]:
sgd_text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('sgd-clf', SGDClassifier(loss='hinge', 
                                                   penalty='l2',
                                                   alpha=1e-3, 
                                                   random_state=13))])

sgd_text_clf = text_clf.fit(X_train, y_train)
sgd_text_clf.score(X_test, y_test)

/Users/Ditwoo/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.82524271844660191